In [32]:
""""
Figaro times collection times script. 
"""
import numpy as np
import pandas as pd
import os
import regex as re
from openpyxl import load_workbook, Workbook
from openpyxl.utils.cell import get_column_letter

exp_names = ["svd_mkl_dc", "svd_figaro"]
exp_comp = {"svd_mkl_dc"}
exp_fig = "svd_figaro"
exp_paths = {"svd_mkl_dc": "/local/scratch/Figaro/figaro-code/dumps/decomp_alg/svd/divide_and_conquer/sing_vals_100/col_major/s_only", "svd_figaro": "/local/scratch/Figaro/figaro-code/dumps/figaro/svd/divide_and_conquer/householder/sing_vals_100/s_onlyt/hread48"}
db_names = ["DBRetailer", "DBFavorita", "DBYelp"]
join_orders = {"DBRetailer": "LocationRoot", "DBFavorita": "StoresRoot", "DBYelp": "BusinessRoot"}
start_per = 10
end_per = 100
per_inc = 10
ohe = False

csv_name = "R.csv"
numeric_const_pattern = r"Relative error is: ([+-]?(\d+(\.\d*)?|\.\d+)([eE][+-]?\d+)?)"
rx = re.compile(numeric_const_pattern)

df_measurement_exps = {}

df_measurement = pd.DataFrame(columns=db_names)

np.set_printoptions(precision=20)
for db_name in db_names:
    join_order = join_orders[db_name]

    if not ohe:
        db_name_per = "{}{}".format(db_name, 100)
    else:
        db_name_per = "{}PK1C{}".format(db_name, 100)


    exp_path = exp_paths["svd_figaro"]
    path_csv = os.path.join(exp_path, db_name_per,
    join_order, csv_name)
    print(path_csv)
    np_figaro = pd.read_csv(path_csv, header=None).astype(float).to_numpy()
    
    exp_path = exp_paths["svd_mkl_dc"]
    path_csv = os.path.join(exp_path, db_name_per,
    join_order, csv_name)
    np_mkl = pd.read_csv(path_csv, header=None).astype(float).to_numpy()
    
    #print(np_figaro)
    #print(np_mkl)
    (len, _) = np.shape(np_mkl)
    print(len)
    dist = per_inc / 100.0 * len
    curDist = 0.0
    print(dist)
    for db_idx, percent in enumerate(range(start_per, end_per + 1, per_inc)):
        next_val = (int)(curDist + dist)
        per_prev_val = (int)(curDist)
        if (percent == 100):
            next_val += 1
        np_fig_per = np_figaro[per_prev_val:next_val]
        np_mkl_per = np_mkl[per_prev_val:next_val]
        np_diff = np_fig_per - np_mkl_per
        
        

        abs_err = np.linalg.norm(np_diff, ord='fro')
        abs_err_comp  = np.linalg.norm(np_mkl_per, ord='fro')
        print("percent", percent, per_prev_val, next_val)
        print(np_diff)
        #print("figaro", np_fig_per, np_fig_per.shape)
        #print("original", np_mkl_per, np_mkl_per.shape)
        #if (percent == 100):
            #print(np_mkl_per)
            #print(np_figaro)
        relative_frob_norm = abs_err / abs_err_comp

        df_measurement.at[percent, db_name] = relative_frob_norm
        curDist += dist
        
dir_out = "ohe" if ohe else "non-ohe"
os.makedirs(dir_out, exist_ok=True)
file_out = os.path.join(dir_out, "sing_values.csv")
df_measurement.to_csv(file_out, sep="\t")


/local/scratch/Figaro/figaro-code/dumps/figaro/svd/divide_and_conquer/householder/sing_vals_100/s_onlyt/hread48/DBRetailer100/LocationRoot/R.csv
43
4.3
percent 10 0 4
[[ 0.0000000000000000e+00]
 [ 3.8743019104003906e-07]
 [ 5.0663948059082031e-07]
 [-1.6391277313232422e-07]]
percent 20 4 8
[[-6.7055225372314453e-07]
 [ 4.3213367462158203e-07]
 [ 1.1175870895385742e-08]
 [ 2.9802322387695312e-08]]
percent 30 8 12
[[-9.3132257461547852e-09]
 [ 1.0244548320770264e-08]
 [-6.4726918935775757e-08]
 [-8.7078660726547241e-08]]
percent 40 12 17
[[ 2.7474015951156616e-08]
 [ 1.7927959561347961e-08]
 [ 1.6065314412117004e-08]
 [-3.4924596548080444e-10]
 [-5.8207660913467407e-10]]
percent 50 17 21
[[3.2014213502407074e-09]
 [0.0000000000000000e+00]
 [8.4983184933662415e-09]
 [3.0413502827286720e-09]]
percent 60 21 25
[[ 1.7113052308559418e-08]
 [ 3.6379788070917130e-11]
 [ 7.2759576141834259e-11]
 [-1.0913936421275139e-11]]
percent 70 25 30
[[ 2.561137080192566e-09]
 [ 8.258211892098188e-10]
 [-3.